In [ ]:
from pyspark.sql import SparkSession
import os

from dotenv import load_dotenv

load_dotenv()

os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages org.apache.hadoop:hadoop-aws:3.3.4,io.delta:delta-spark_2.12:3.3.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.4,org.apache.kafka:kafka-clients:3.9.0,org.apache.spark:spark-avro_2.12:3.5.1 pyspark-shell"

# Initialize Spark session with Delta Lake and MinIO support
spark = (SparkSession.builder \
    .appName("DeltaLakeWithMinIO") \
    ## Delta
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    #Hive Catalog
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    ## Optimize Delta
    .config("delta.autoOptimize.optimizeWrite", "true") \
    .config("delta.autoOptimize.autoCompact", "true") \
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
    ## MinIO
    .config("spark.hadoop.fs.s3a.endpoint", os.getenv("MINIO_ENDPOINT")) \
    .config("spark.hadoop.fs.s3a.access.key", os.getenv("MINIO_ACCESS_KEY")) \
    .config("spark.hadoop.fs.s3a.secret.key", os.getenv("MINIO_SECRET_KEY")) \
    .config('spark.hadoop.fs.s3a.attempts.maximum', "3") \
    .config('spark.hadoop.fs.s3a.connection.timeout', "10000") \
    .config('spark.hadoop.fs.s3a.connection.establish.timeout', "5000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3n.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate())

In [ ]:
def get_spark_schema():
    with open("../src/schemas/control_power.json") as file:
        avro_schema_str = file.read()

    return avro_schema_str

In [ ]:
from pyspark.sql.functions import current_timestamp, expr
from pyspark.sql.avro.functions import from_avro

# Kafka Configuration
kafka_broker = "kafka-cpc.certi.org.br:31289"
topic_name = "control_power-avro"

# Read data from Kafka
kafka_stream = spark.read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_broker) \
    .option("subscribe", topic_name) \
    .option("startingOffsets", "earliest") \
    .load()

# Deserialize Kafka value (JSON string) into columns
parsed_stream = kafka_stream.select(
        "timestamp",
        from_avro(
            expr("substring(value, 6, length(value)-5)"),
            get_spark_schema()
        ).alias("data")
    ) \
    .select("timestamp", "data.*")

parsed_stream_with_timestamp = parsed_stream.withColumn("landing_timestamp", current_timestamp())

# Output the parsed stream for verification
parsed_stream_with_timestamp.printSchema()


In [ ]:
df = parsed_stream_with_timestamp.toPandas()
df.head()

In [ ]:
import pandas as pd

df["source_timestamp"] = pd.to_datetime(df["source_timestamp"]).dt.tz_convert("America/Sao_Paulo")
df["landing_timestamp"] = df["landing_timestamp"].dt.tz_localize("America/Sao_Paulo")
df["timestamp"] = df["timestamp"].dt.tz_localize("America/Sao_Paulo")
df.head()

In [ ]:
df["source_kafka_latency"] = df["timestamp"] - df["source_timestamp"]
df["source_kafka_latency"].describe()

In [ ]:
import matplotlib.pyplot as plt

# Convert timedelta to seconds for better visualization
df["source_kafka_latency_sec"] = df["source_kafka_latency"].dt.total_seconds()

# Plot histogram
plt.figure(figsize=(10, 5))
plt.hist(df["source_kafka_latency_sec"], bins=50, edgecolor="black")
plt.xlabel("Latency (seconds)")
plt.ylabel("Frequency")
plt.title("Distribution of Kafka Source Latency")
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(8, 4))
plt.boxplot(df["source_kafka_latency_sec"], vert=False)
plt.xlabel("Latency (seconds)")
plt.title("Box Plot of Kafka Source Latency")
plt.grid(True)
plt.show()


In [ ]:
spark.sql("SELECT timestamp, landing_timestamp, parsed_value.* FROM delta.`s3a://lakehouse/delta/raw_control_power-avro`;").show(5)

In [ ]:
df_datalake = spark.sql("SELECT timestamp, landing_timestamp, parsed_value.* FROM delta.`s3a://lakehouse/delta/raw_control_power-avro`;").toPandas()

In [ ]:
import pandas as pd

df_datalake["source_timestamp"] = pd.to_datetime(df_datalake["source_timestamp"]).dt.tz_convert("America/Sao_Paulo")
df_datalake["landing_timestamp"] = df_datalake["landing_timestamp"].dt.tz_localize("America/Sao_Paulo")
df_datalake["timestamp"] = df_datalake["timestamp"].dt.tz_localize("America/Sao_Paulo")

df_datalake["source_kafka_latency"] = df_datalake["timestamp"] - df_datalake["source_timestamp"]
df_datalake["source_kafka_latency"].describe()

In [ ]:
df_datalake["kafka_landing_latency"] = df_datalake["landing_timestamp"] - df_datalake["timestamp"]
df_datalake["kafka_landing_latency"].describe()